In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys

BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.Ticker as Ticker
import method.SaleRules as _sr
import method.algofuncs as _af
path = os.getcwd()

Loading BokehJS ...

In [3]:
path = os.getcwd()
class FollowTheTrend(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        opens = self.data.Open
        volumes = self.data.Volume
        if len(self.data.Volume) > 5:
            last_price = prices[-1]
            if self.buy_price == 0 and Ticker.isStockOut(prices):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and (_sr.takeProfitByPercent(5, last_price, self.buy_price) or _sr.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [5]:
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
vn30_ticker = _af.getListVN30()
reports = [['Ticker', 'Avg. Trade [%]', 'Equity Final [$]', 'Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Trades']]
for ticker_id in vn30_ticker:
    ticker = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2010-01-01', '2020-10-05')
    bt = Backtest(ticker, FollowTheTrend, commission=.005, exclusive_orders=False)
    stats = bt.run()
    if stats['_trades'].shape[0]:
        reports = np.append(reports, [[ticker_id, stats['Avg. Trade [%]'], stats['Equity Final [$]'], stats['Return [%]'], stats['Sharpe Ratio'], stats['Win Rate [%]'], stats['_trades'].shape[0]]], axis=0)
ls = pd.DataFrame(data=reports[1:,1:],index=reports[1:,0],columns=reports[0,1:])
ls_s = ls.sort_values(by=['Avg. Trade [%]'], ascending=False, na_position='first')
ls_s


,Avg. Trade [%],Equity Final [$],Return [%],Sharpe Ratio,Win Rate [%],Trades
VPB,2.2605700689254826,13072.05419900002,30.7205419900002,0.5820326999034867,75.0,12
VNM,1.4274404555715048,22122.53138200013,121.22531382000128,0.44963602555925936,69.64285714285714,56
FPT,1.3794916159445947,20941.766833500136,109.41766833500137,0.4019537660171739,74.07407407407408,54
GAS,1.0548409790351654,13087.318773500028,30.873187735000286,0.2172468105303471,72.0,25
NVL,1.040241541027287,11438.195088500015,14.38195088500015,0.2571220136713763,69.23076923076923,13
DHG,0.9352970132219918,14589.316238500069,45.893162385000686,0.1990486008697148,69.04761904761905,42
SBT,0.8380112537547779,14977.518470000072,49.77518470000072,0.21945323637006342,69.38775510204081,49
MBB,0.8116859569092805,13162.236632000038,31.622366320000385,0.18979930376697937,67.64705882352942,34
CTD,0.5958471248238562,12594.036751000054,25.940367510000545,0.14083481830572997,61.53846153846154,39
VRE,0.5276836805393259,10213.681415500014,2.1368141550001383,0.04009870880454503,69.23076923076923,13


60